### Get the Personalize boto3 Client

In [19]:
import boto3

import json
import numpy as np
import pandas as pd
import time

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Specify a Bucket and Data Output Location

In [20]:
bucket = "nicolai-personalize"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"  # replace with a name that you want to save the dataset under

### Download, Prepare, and Upload Training Data

#### Download and Explore the Dataset

In [15]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2019-09-05 16:46:45--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-100k.zip’ not modified on server. Omitting download.

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflat

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


#### Prepare and Upload Data

In [16]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Create Schema

In [9]:
!aws personalize delete-schema --schema-arn arn:aws:personalize:eu-west-1:811065773072:schema/Personalize-MovieLens-schema

schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "Personalize-MovieLens-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))


An error occurred (ResourceInUseException) when calling the DeleteSchema operation: The provided schema is being referenced by one or more datasets


ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateSchema operation: Another resource with Arn arn:aws:personalize:eu-west-1:811065773072:schema/Personalize-MovieLens-schema already exists.

### Create and Wait for Dataset Group

#### Create Dataset Group

In [18]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "Personalize-MovieLens-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:eu-west-1:811065773072:dataset-group/Personalize-MovieLens-dataset-group",
  "ResponseMetadata": {
    "RequestId": "3e638de5-7cae-4f0a-9d52-3f3e04851b15",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 16:46:57 GMT",
      "x-amzn-requestid": "3e638de5-7cae-4f0a-9d52-3f3e04851b15",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Group to Have ACTIVE Status

In [21]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

DatasetGroup: ACTIVE


### Create Dataset

In [22]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "Personalize-MovieLens-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:eu-west-1:811065773072:dataset/Personalize-MovieLens-dataset-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "364003a6-da12-495f-9ac8-e464ca8dac35",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 16:48:04 GMT",
      "x-amzn-requestid": "364003a6-da12-495f-9ac8-e464ca8dac35",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Prepare, Create, and Wait for Dataset Import Job

#### Attach Policy to S3 Bucket

In [23]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '83337F3415D71EFC',
  'HostId': 'SOMARFxYKZvnC/lsNcMG2//3pTPX/hILsPNNrkbg2/zQK7M8gm8dGa490Jhw2KlP4ixp1P8l9bs=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'SOMARFxYKZvnC/lsNcMG2//3pTPX/hILsPNNrkbg2/zQK7M8gm8dGa490Jhw2KlP4ixp1P8l9bs=',
   'x-amz-request-id': '83337F3415D71EFC',
   'date': 'Thu, 05 Sep 2019 16:48:08 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Create Personalize Role

In [24]:
iam = boto3.client("iam")

role_name = "PersonalizeRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    )

    # AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
    # if you would like to use a bucket with a different name, please consider creating and attaching a new policy
    # that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
    policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = policy_arn
    )

    time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate
    role_arn = create_role_response["Role"]["Arn"]
except:
    print("Role already exists")
    get_role_response = iam.get_role(
        RoleName = role_name)
    role_arn = get_role_response["Role"]["Arn"]
    
print(role_arn)

Role already exists
arn:aws:iam::811065773072:role/PersonalizeRole


#### Create Dataset Import Job

In [25]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "Personalize-MovieLens-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:eu-west-1:811065773072:dataset-import-job/Personalize-MovieLens-dataset-import-job",
  "ResponseMetadata": {
    "RequestId": "7415ec72-cfb2-47a6-b373-96db855c10aa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 16:48:17 GMT",
      "x-amzn-requestid": "7415ec72-cfb2-47a6-b373-96db855c10aa",
      "content-length": "128",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Dataset Import Job to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


### Select Recipe

In [6]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 46, 30, 358000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 46, 30, 357000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 46, 30, 357000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

### Create and Wait for HRNN Solution

#### Create Solution

In [28]:
create_solution_response = personalize.create_solution(
    name = "Personalize-MovieLens-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-solution",
  "ResponseMetadata": {
    "RequestId": "cbd3d12f-77eb-4cf4-96cb-95dfd2173f63",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 17:00:31 GMT",
      "x-amzn-requestid": "cbd3d12f-77eb-4cf4-96cb-95dfd2173f63",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [29]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-solution/b4012231",
  "ResponseMetadata": {
    "RequestId": "eccb007e-5e5a-4f95-ab4f-2474148f76b0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 17:00:35 GMT",
      "x-amzn-requestid": "eccb007e-5e5a-4f95-ab4f-2474148f76b0",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

#### Get Metrics of Solution

In [32]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-solution/b4012231",
  "metrics": {
    "coverage": 0.2714,
    "mean_reciprocal_rank_at_25": 0.0464,
    "normalized_discounted_cumulative_gain_at_10": 0.0611,
    "normalized_discounted_cumulative_gain_at_25": 0.0884,
    "normalized_discounted_cumulative_gain_at_5": 0.0413,
    "precision_at_10": 0.0114,
    "precision_at_25": 0.0091,
    "precision_at_5": 0.0114
  },
  "ResponseMetadata": {
    "RequestId": "05a72ccc-f115-4652-8692-a8a784ccd2ef",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 17:38:46 GMT",
      "x-amzn-requestid": "05a72ccc-f115-4652-8692-a8a784ccd2ef",
      "content-length": "414",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create AutoML-based Solution

In [21]:
create_automl_solution_response = personalize.create_solution(
    name = "Personalize-MovieLens-AutoML-solution",
    datasetGroupArn = dataset_group_arn,
    performAutoML = True
)

automl_solution_arn = create_automl_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

NameError: name 'dataset_group_arn' is not defined

NameError: name 'dataset_group_arn' is not defined

#### Create Solution Version

In [22]:
solution_arn = "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-AutoML-solution"

create_automl_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

automl_solution_version_arn = create_automl_solution_version_response['solutionVersionArn']
print(json.dumps(create_automl_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-AutoML-solution/1ceb5be9",
  "ResponseMetadata": {
    "RequestId": "3131d0aa-9c89-4395-b0b3-3ccd7820f65d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 10 Sep 2019 17:33:41 GMT",
      "x-amzn-requestid": "3131d0aa-9c89-4395-b0b3-3ccd7820f65d",
      "content-length": "123",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-AutoML-solution/1ceb5be9",
  "ResponseMetadata": {
    "RequestId": "3131d0aa-9c89-4395-b0b3-3ccd7820f65d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 10 Sep 2019 17:33:41 GMT",
      "x-amzn-requestid": "3131d0aa-9c89-4395-b0b3-3ccd7820f65d",
      "content-length": "123",
      "connection": "keep

#### Wait for Solution Version to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_automl_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = automl_solution_version_arn
    )
    status = describe_automl_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


#### Get Metrics of Solution

In [18]:
import datetime

get_automl_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = automl_solution_version_arn
)

get_automl_solution_details_response = personalize.describe_solution_version(
    solutionVersionArn = automl_solution_version_arn
)

print(json.dumps(get_automl_solution_metrics_response, indent=2))

def conv(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
 
print(json.dumps(get_automl_solution_details_response, indent=2, default=conv))


{
  "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-solution/c06b0760",
  "metrics": {
    "coverage": 0.27,
    "mean_reciprocal_rank_at_25": 0.0699,
    "normalized_discounted_cumulative_gain_at_10": 0.0846,
    "normalized_discounted_cumulative_gain_at_25": 0.1122,
    "normalized_discounted_cumulative_gain_at_5": 0.0697,
    "precision_at_10": 0.0125,
    "precision_at_25": 0.0095,
    "precision_at_5": 0.0159
  },
  "ResponseMetadata": {
    "RequestId": "fba22eed-b3fb-4b2f-a079-72f354f57fac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 10 Sep 2019 17:28:43 GMT",
      "x-amzn-requestid": "fba22eed-b3fb-4b2f-a079-72f354f57fac",
      "content-length": "412",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionVersion": {
    "solutionVersionArn": "arn:aws:personalize:eu-west-1:811065773072:solution/Personalize-MovieLens-solution

### Create and Wait for Campaign

#### Create Campaign

In [37]:
create_campaign_response = personalize.create_campaign(
    name = "Personalize-MovieLens-campaign",
    solutionVersionArn = automl_solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:eu-west-1:811065773072:campaign/Personalize-MovieLens-campaign",
  "ResponseMetadata": {
    "RequestId": "3bf9a88a-b9ce-40e1-8545-692f8afc08eb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 05 Sep 2019 18:22:47 GMT",
      "x-amzn-requestid": "3bf9a88a-b9ce-40e1-8545-692f8afc08eb",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [42]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(30)

Campaign: ACTIVE


### Get Recommendations

#### Select a User and an Item

In [24]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], header=None, encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 144
ITEM: Best Men (1997)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
...,...,...
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


#### Call GetRecommendations

In [25]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

NameError: name 'campaign_arn' is not defined